In [36]:
# Import libraries
import gymnasium as gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [37]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [38]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

In [39]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [40]:
def train(q, q_target, memory, optimizer, device):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        # move samples to device
        s = s.to(device)
        a = a.to(device)
        r = r.to(device)
        s_prime = s_prime.to(device)
        done_mask = done_mask.to(device)
        
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [41]:
def main():
    env = gym.make('CartPole-v1')
    q = Qnet().to(device)
    q_target = Qnet().to(device)
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    for n_epi in range(10000):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s, _ = env.reset()
        done = False
        
        while not done:
            a = q.sample_action(torch.from_numpy(s).float().to(device), epsilon)      
            s_prime, r, done, truncated, info = env.step(a)
            done_mask = 0.0 if done else 1.0
            memory.put((s,a,r/100.0,s_prime, done_mask))
            s = s_prime
            
            score += r
            if done:
                break
            
        if memory.size()>2000:
            train(q, q_target, memory, optimizer, device)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
    env.close()

In [42]:
if __name__ == '__main__':
    print(device)
    main()

cuda
n_episode :20, score : 10.8, n_buffer : 215, eps : 7.9%
n_episode :40, score : 9.2, n_buffer : 400, eps : 7.8%
n_episode :60, score : 9.3, n_buffer : 586, eps : 7.7%
n_episode :80, score : 9.9, n_buffer : 785, eps : 7.6%
n_episode :100, score : 9.7, n_buffer : 978, eps : 7.5%
n_episode :120, score : 9.8, n_buffer : 1173, eps : 7.4%
n_episode :140, score : 9.9, n_buffer : 1371, eps : 7.3%
n_episode :160, score : 9.7, n_buffer : 1564, eps : 7.2%
n_episode :180, score : 9.8, n_buffer : 1760, eps : 7.1%
n_episode :200, score : 9.6, n_buffer : 1952, eps : 7.0%
n_episode :220, score : 9.9, n_buffer : 2150, eps : 6.9%
n_episode :240, score : 9.3, n_buffer : 2337, eps : 6.8%
n_episode :260, score : 10.0, n_buffer : 2537, eps : 6.7%
n_episode :280, score : 11.5, n_buffer : 2767, eps : 6.6%
n_episode :300, score : 14.8, n_buffer : 3063, eps : 6.5%
n_episode :320, score : 15.4, n_buffer : 3371, eps : 6.4%
n_episode :340, score : 111.7, n_buffer : 5605, eps : 6.3%
n_episode :360, score : 152.

KeyboardInterrupt: 